<a href="https://colab.research.google.com/github/yini2105/hospital_management/blob/main/Hospital_Management_Yini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import glob # finds files using patterns
import os # paths/folders

import kagglehub
data = kagglehub.dataset_download("jaderz/hospital-beds-management")

new = sqlite3.connect("/content/hospital_beds.db") # create new database

csv_files = glob.glob(os.path.join(data, "**", "*.csv"), recursive=True) # searches to match patterns and combine them as much as possible

for files in csv_files:
    table_names = files.split("/")[-1].replace(".csv", "") # splits the path string and shoose the last to replace
    df = pd.read_csv(files) # now we can read it finally
    df.to_sql(table_names, new, if_exists="replace", index=False) # data to SQL -> save table into database

def sql(query):
    return pd.read_sql_query(query, new) # run the SQl on database and return table

print("Ready --> type SQL in the next cell.")


100%|██████████| 46.5k/46.5k [00:00<00:00, 29.1MB/s]

Extracting files...


Ready --> type SQL in the next cell.


In [ ]:
sql("""
  SELECT service, count(*) AS patient_count
  FROM patients
  GROUP BY service
  ORDER BY patient_count DESC;
""")

,service,patient_count
0,emergency,263
1,surgery,254
2,general_medicine,242
3,ICU,241


In [ ]:
sql("""
SELECT
    service,
    ROUND(1.0 * SUM(patients_admitted)/SUM(available_beds), 2) AS bed_pressure
FROM services_weekly
GROUP BY service
ORDER BY bed_pressure DESC;
""")

,service,bed_pressure
0,emergency,1.00
1,general_medicine,0.97
2,surgery,0.86
3,ICU,0.84


In [ ]:
sql("""
SELECT
    service,
    ROUND(1.0 * SUM(patients_refused) / SUM(patients_request), 2) AS refusal_rate
FROM services_weekly
GROUP BY service
ORDER BY refusal_rate DESC;
""")


,service,refusal_rate
0,emergency,0.81
1,general_medicine,0.45
2,surgery,0.25
3,ICU,0.18


In [ ]:
sql("""
SELECT
  week,
  month,
  service,
  available_beds,
  patients_request,
  patients_admitted,
  patients_refused,
  ROUND(1.0 * patients_admitted / available_beds, 2) AS bed_pressure,
  ROUND(1.0 * patients_refused / patients_request, 2) AS refusal_rate,
  event
FROM services_weekly
WHERE service = 'emergency'
ORDER BY refusal_rate DESC
LIMIT 15;
""")


,week,month,service,available_beds,patients_request,patients_admitted,patients_refused,bed_pressure,refusal_rate,event
0,5,2,emergency,25,388,25,363,1.0,0.94,flu
1,12,3,emergency,28,347,28,319,1.0,0.92,flu
2,8,2,emergency,26,240,26,214,1.0,0.89,flu
3,22,6,emergency,16,129,16,113,1.0,0.88,none
4,43,11,emergency,21,171,21,150,1.0,0.88,none
5,48,12,emergency,20,165,20,145,1.0,0.88,none
6,10,3,emergency,17,130,17,113,1.0,0.87,none
7,14,4,emergency,15,97,15,82,1.0,0.85,none
8,11,3,emergency,16,97,16,81,1.0,0.84,donation
9,13,4,emergency,23,142,23,119,1.0,0.84,none


In [ ]:
sql("""
SELECT
  event,
  COUNT(*) AS weeks,
  ROUND(AVG(1.0 * patients_admitted / available_beds), 2) AS avg_bed_pressure,
  ROUND(AVG(1.0 * patients_refused / patients_request), 2) AS avg_refusal_rate
FROM services_weekly
WHERE service = 'emergency'
GROUP BY event
ORDER BY avg_refusal_rate DESC;
""")


,event,weeks,avg_bed_pressure,avg_refusal_rate
0,flu,5,1.0,0.88
1,none,39,1.0,0.77
2,donation,4,1.0,0.73
3,strike,4,1.0,0.63


In [ ]:
sql("""
SELECT
  service,
  COUNT(*) AS flu_weeks,
  ROUND(AVG(1.0 * patients_admitted / available_beds), 2) AS avg_bed_pressure,
  ROUND(AVG(1.0 * patients_refused / patients_request), 2) AS avg_refusal_rate
FROM services_weekly
WHERE event = 'flu'
GROUP BY service
ORDER BY avg_refusal_rate DESC;
""")


,service,flu_weeks,avg_bed_pressure,avg_refusal_rate
0,emergency,5,1.0,0.88
1,general_medicine,6,1.0,0.74
2,surgery,3,1.0,0.66
3,ICU,5,1.0,0.33


In [ ]:
sql("""
SELECT
  week,
  month,
  service,
  available_beds,
  patients_request,
  patients_admitted,
  patients_refused,
  ROUND(1.0 * patients_admitted / available_beds, 2) AS bed_pressure,
  ROUND(1.0 * patients_refused / patients_request, 2) AS refusal_rate,
  patient_satisfaction,
  staff_morale,
  event
FROM services_weekly;
""")


,week,month,service,available_beds,patients_request,patients_admitted,patients_refused,bed_pressure,refusal_rate,patient_satisfaction,staff_morale,event
0,1,1,emergency,32,76,32,44,1.00,0.58,67,70,none
1,1,1,surgery,45,130,45,85,1.00,0.65,83,78,flu
2,1,1,general_medicine,37,201,37,164,1.00,0.82,97,43,flu
3,1,1,ICU,22,31,22,9,1.00,0.29,84,91,flu
4,2,1,emergency,28,169,28,141,1.00,0.83,75,64,none
...,...,...,...,...,...,...,...,...,...,...,...,...
203,51,12,ICU,18,15,15,0,0.83,0.00,87,80,none
204,52,12,emergency,26,130,26,104,1.00,0.80,88,63,none
205,52,12,surgery,53,39,39,0,0.74,0.00,82,50,none
206,52,12,general_medicine,65,225,65,160,1.00,0.71,82,40,flu
